## 海龟交易法则


**导语**：介绍非常著名的交易系统—海龟交易法则。


#### 一、策略阐述


**1.海龟交易法则的由来**

　　Riachard Dennis是七八十年代著名的期货投机商，是一位具有传奇色彩的人物，在多年的投机生涯中，Dennis出尽风头，给人的感觉是常常可以在最低点买进，然后在最高峰反手卖空。<br>

　　他相信优秀的交易员是后天培养而非天生的。在1983年12月，他招聘了23名新人，昵称为海龟，并对这些交易员进行了一个趋势跟踪交易策略培训。随后给予每个新人100万美元的初始资金。经5年的运作，大部分“海龟”的业绩非常惊人，其中最好的业绩达到1.72亿美元。多年后，海龟交易法则公布于世，我们才有幸看到曾名噪一时的完整的海龟交易法则。<br>

**2.海龟交易法则介绍**


　　A.市场与标的<br>

　　　海龟交易法则应用于流动性高的市场中，选择交易量较大的标的进行交易。本文以沪深300指数ETF为标的，构建海龟交易法则。<br>

　　B、仓位：仓位是海龟交易系统最核心的部分，通过ATR真实波幅指标来管理仓位。<br>

　　　第一步：计算True Range，简称TR。<br>

　　　　TR = Max ( H−L , H−P , P−L ) ，其中H为当日日内最高价，L为当日日内最低价，P为前一日收盘价。<br>

　　　第二步：计算ATR<br>

　　　　ATR = mean ( TR , 20 ) , 即计算过去20天的TR的平均值，ATR是TR的移动平均值<br>

　　　第三步：计算unit（法则的交易单位）<br>

　　　　Unit = (value x 1% ) / ATR ，其中value x 1%即为总资产的1%，考虑到国内最小变化量是0.01元，1手是100股，所以1ATR即为持1股股票的资产最大变动，那么买入1Unit单位的股票，使得总资产当天震幅不超过1% 。<br>

　　C.开仓入市<br>

　　　海龟交易法则分两个交易系统，两者都为分钟回测，即盘中交易：<br>

　　系统一：<br>

　　　I、若当前价格高于过去20日的最高价，则买入一个Unit<br>
　　　II、加仓：若股价在上一次买入的基础上上涨了0.5ATR，则加仓一个Unit。<br>

　　系统二：<br>
　
　　　I、若当前价格高于过去55日的最高价，则买入一个Unit<br>
　　　II、加仓：若股价在上一次买入的基础上上涨了0.5N，则加仓一个Unit。<br>

　　举例：若某只股票A的ATR为1，20日最高价为40。<br>
　　　　　则当股价突破40时买入一个Unit，当股价突破40+0.5×1=40.5时加仓一个Unit。<br>
　　　　　当股价突破40.5+0.5×1=41时加仓一个Unit。<br>


　　D.止损：海龟交易法则规定，当价格比最后一次买入价格下跌2ATR时，则卖出全部头寸止损。<br>


　　E.止盈：两个系统分别采用不同参数来止盈。<br>


　　系统一：当股价跌破10日内最低价格时，清仓结束交易<br>

　　系统二、当股价跌破20日内最低价格时，清仓结束交易<br>


　　考虑到系统一与系统二的差异性集中在参数上，本篇内容实现系统二，来向大家展示海龟交易法则。<br>

　　以下为策略实现的基本信息：<br>

　　策略实现难度：2<br>
　　实现过程中所需要用到的API函数，ps:通过MindGo量化交易平台API文档快速掌握：

|需要用到的API函数|功能
|---|---|
|account.portfolio_value|获取账户总资产
|set_benchmark()|设置基准指数


#### 二、代码示意图

![](http://u.thsi.cn/fileupload/data/Sns/2018/1bcbde1579eecadc1b504b704af34c8e.png)


#### 三、编写释义


　　本策略的编写难点在于理解海龟交易法则的运行逻辑，以下是海龟法则运行逻辑的梳理：<br>
　　编写海龟交易法则的时候，建议采用主干+枝干的思路。
![](http://u.thsi.cn/fileupload/data/Sns/2018/1e37464efd84d6c142c52a46d1f482e8.png)
![](http://u.thsi.cn/fileupload/data/Sns/2018/a4f1eb5a431354f4d740fb3be55e42a5.png)


#### 四、最终结果

策略回测区间：2014.01.01-2018.01.29<br>
回测资金：1000000<br>
回测频率：分钟<br>
回测结果：红色曲线为策略收益率曲线，蓝色曲线为对应的基准指数收益率曲线<br>
![](http://u.thsi.cn/fileupload/data/Sns/2018/2940b8b27c2f93216f28f5228962d299.png)

策略源代码：

In [ ]:
import pandas as pd 
import numpy as np 
#===========================初始化账户==================================== 
def initialize(account):
    account.security = '159919.OF'#确定交易标的
    set_benchmark(account.security)
    account.ART = 0#储存ATR的值，每个交易日更新一次
    account.unit = 0#买卖单位的储存变量
    account.steam = False#交易系统
    account.price = 0 #记录系统的买入价，以便加仓和离市
#======================盘前运行============================
def before_trading_start(account,data):  
    #更新n值
    ATR=get_ATR(account,data)
    #获取账户总资产
    value=account.portfolio_value
    # 依本策略，计算一个单位的unit为多少股,以便后续下单交易,注意一共两个系统，因此需要除以2
    account.unit = (value*0.01)/account.ATR
    if account.unit<100:
        log.info('一个unit单位的股数不满1手，无法下单！')
    
#======================设置买卖条件，分钟回测============================
def handle_data(account,data):
      #====================系统1================================
    #系统是否需要开启运作
    if account.steam == False:
        #获取开启系统的结果
        account.steam = steam(account,data,55)
        if account.steam == False:
            pass
        else:
            order(account.security,account.unit)
            #买入后记录当前价位，以便加仓和离市
            log.info('系统开启')
            nowclose = history(account.security, ['close'], 1, '1m', False, 'pre', is_panel=1)['close']
            account.price = nowclose[0]
    #系统已经开启运作
    if account.steam == True:
        #获取进行加仓结果
        signal=addtrade(account,data)
        #执行结果加仓
        if signal=='buy':
            log.info('系统加仓')
            nowclose = history(account.security, ['close'], 1, '1m', False, 'pre', is_panel=1)['close']
            order(account.security,account.unit)
            #买入后记录当前价位，以便加仓和离市
            account.price1 = nowclose[0]
        else:
            pass
        #获取止盈结果
        signal=down(account,data)
        #执行止盈，关闭系统
        if signal=='sell':
            log.info('系统1止盈')
            order_target(account.security,0)
            #止盈后情况价位记录
            account.price = 0
            #关闭系统
            account.steam = False
    #离场结果判断
    if account.steam==True:
        #获取离场结果
        signal=giveuptrade(account,data,20)
        #执行离场，关闭系统
        if signal=='sell':
            log.info('系统1离场')
            order_target(account.security,0)
            #离场后情况价位记录
            account.price=0
            #关闭系统
            account.steam=False
#=================判断是否离场函数============================
def giveuptrade(account,data,n):
    #根据系统来获取相应数据长度
    close = history(account.security, ['low'], n, '1d', False, 'pre', is_panel=1)['low']
    close_min=min(close)
    nowclose = history(account.security, ['close'], 1, '1m', False, 'pre', is_panel=1)['close']
    #最新价格突破过去N日最大收盘价，即为突破，开启系统
    if nowclose[0]<close_min:
        return 'sell'
    else:
        return None
#==================判断是否止盈函数=================================
def down(account,data):
    nowclose = history(account.security, ['close'], 1, '1m', False, 'pre', is_panel=1)['close']
    n=account.ATR
    TP=account.price-2*n
    if nowclose[0]<TP:
        return 'sell'
    else:
        return None

#==================判断是否加仓函数=================================
def addtrade(account,data):
    nowclose = history(account.security, ['close'], 1, '1m', False, 'pre', is_panel=1)['close']
    n=account.ATR
    TP=account.price+n/2
    if nowclose[0]>TP:
        return 'buy'
    else:
        return None
    
#==================判断系统开启函数=================================
def steam(account,data,n):
    close = history(account.security, ['close'], n, '1d', False, 'pre', is_panel=1)['close']
    close_max=max(close)
    nowclose = history(account.security, ['close'], 1, '1m', False, 'pre', is_panel=1)['close']
    #最新价格突破过去N日最大收盘价，即为突破，开启系统
    if nowclose[0]>close_max:
        return True
    else:
        return False
#==================计算n值的函数=================================
def get_ATR(account,data):
    #由于用到前20个交易日的n值，ATR计为过去20日的TR均值
    price = history(account.security, ['close','high','low'], 21, '1d', False, 'pre', is_panel=1)
    h = price['high'].iloc[1:] #最高价，获取21个需弃掉第一个
    l= price['low'].iloc[1:]#最低价，获取21个需弃掉第一个
    rc = price['close'].shift().iloc[1:]#昨日收盘价，获取21个需弃掉第一个
    #shift()操作专门是用于获取前收盘价数据的
    tr_list = []
    for i in range(0,20,1):
        h = price['high'].iloc[i]
        l = price['low'].iloc[i]
        rc = price['close'].iloc[i]
        TR = max(h-l, h-rc, rc-l)
        tr_list.append(TR)
    ATR=np.mean(tr_list)
    account.ATR=ATR
    return account.ATR

In [ ]:
import numpy as np
import matplotlib.pyplot as plt 
from matplotlib.finance import candlestick2_ohlc
import datetime
data=get_price(['000300.SH'], None, '20171110', '1d', ['open','high','low','close'], True, None, 200, is_panel=0)
data=data['000300.SH']
#时间转化格式
time=data.index
t=[]
for x in time:
    x=str(x).split()[0]
    x=x.split('-')
    x=x[0]+x[1]+x[2]
    x=int(x)
    t.append(x)
#画图数据
time=t
open1=list(data['open'])
high1=list(data['high'])
low1=list(data['low'])
close1=list(data['close'])
#画图
fig,ax = plt.subplots(figsize = (20,8),facecolor='pink')
fig.subplots_adjust() 
plt.xticks()  
plt.yticks()  
plt.title("沪深300K线走势图")  
plt.ylabel("股指")  
ticks = ax.set_xticks(range(1,200,40))
labels = ax.set_xticklabels([time[0],time[40],time[80],time[120],time[160]]) 
candlestick2_ohlc(ax,open1,high1,low1,close1,width=0.6,colorup='red',colordown='green')
#支撑线
plt.plot([75,200],[3316,3954],'g',linewidth=10)
# 红星：回踩1
plt.plot(75, 3316, 'r*', markersize = 40.0,label='趋势线')
plt.annotate(r'二次低位', xy=(75, 3316),
    xycoords='data', xytext=(-90, -50),
    textcoords='offset points', fontsize=26,
    arrowprops=dict(arrowstyle="->", connectionstyle="arc3,rad=.2"))
# 红星：回踩2
plt.plot(140, 3650, 'r*', markersize = 40.0)
plt.annotate(r'止跌，形成趋势线', xy=(140, 3650),
    xycoords='data', xytext=(-90, -50),
    textcoords='offset points', fontsize=26,
    arrowprops=dict(arrowstyle="->", connectionstyle="arc3,rad=.2"))
# 红星：回踩3
plt.plot(172, 3800, 'r*', markersize = 40.0)
plt.annotate(r'回踩趋势线', xy=(172, 3800),
    xycoords='data', xytext=(-90, -50),
    textcoords='offset points', fontsize=26,
    arrowprops=dict(arrowstyle="->", connectionstyle="arc3,rad=.2"))
#MA5
data['ma5']=pd.rolling_mean(data['close'],5)
plt.plot(list(data['ma5']),label='五日均线')
#MA10
data['ma10']=pd.rolling_mean(data['close'],10)
plt.plot(list(data['ma10']),label='十日均线')
#MA20
data['ma20']=pd.rolling_mean(data['close'],20)
plt.plot(list(data['ma20']),label='二十日均线')
#MA30
data['ma30']=pd.rolling_mean(data['close'],30)
plt.plot(list(data['ma30']),label='三十日均线')
#MA60
data['ma60']=pd.rolling_mean(data['close'],60)
plt.plot(list(data['ma60']),label='六十日均线')
plt.legend()
print('沪深300走势图分析')